In [ ]:
#ML_answer_1

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
####reading data####
df=pd.read_csv('instagram_reach.csv')

In [ ]:
df.head()

In [ ]:
###checking for null_data
df.isnull().sum()

In [ ]:
###checking for duplicate data####
df.duplicated().sum()

In [ ]:
###converting time_since_posted into integer type###
df['Time since posted']=df['Time since posted'].apply((lambda x:x.split()))

time_hrs=[]
for i in df['Time since posted']:
    i.remove('hours')
    time_hrs.extend(i)
df['time']=time_hrs
df['time']=df['time'].astype('int64')

In [ ]:
##sorting numeric and categorical data###
num_list=[]
cat_list=[]
for i in df.columns:
    if df[i].dtypes!='O':
        num_list.append(i)
    else:
        cat_list.append(i)
df1=df[num_list]
df2=df[cat_list]

In [ ]:
df1.drop('S.No',axis=1,inplace=True)

In [ ]:
####checking co_relation between numeric variables
df1.corr()

In [ ]:
##plotting heatmap##
fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(df1.corr(), center=0, annot=True)

In [ ]:
##plotting toc check outliers##
for i in df1:
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    sns.boxplot(df1[i])
    plt.title(i)

In [ ]:
def outliers_detection(dataframe):
    for i in dataframe.columns:
        q1=dataframe[i].quantile(0.25)
        q3=dataframe[i].quantile(0.75)
        iqr=q3-q1
        lower_limit=q1-(1.5*iqr)
        upper_limit=q3+(1.5*iqr)
        print(i,lower_limit,upper_limit)
outliers_detection(df1)

In [ ]:
##plotting distribution plots to check distribution of data in each numeric_column###
for i in df1.columns:
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    sns.distplot(df1[i])
    plt.title(i)
    

In [ ]:
##removing outliers##

df1.loc[(df1['Followers']>2613.375),['Followers']]=2613.375
df1.loc[(df1['Followers']<-1163.625),['Followers']]=df1['Followers'].min()

df1.loc[(df1['time']<0.5),['time']]=0.5
df1.loc[(df1['time']>4.5),['time']]=4.5

df1.loc[df1['Likes']<-21.5]=df1['Likes'].min()
df1.loc[df1['Likes']>86.5]=86.5

In [ ]:
df1.describe()

In [ ]:
sns.regplot(x='Followers',y='Likes',data=df1)

In [ ]:
sns.regplot(x='time',y='Likes',data=df1)

In [ ]:
x=df1.iloc[:,0:2]

In [ ]:
y=df1['Likes']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=25)

In [ ]:
from sklearn.preprocessing import PowerTransformer

In [ ]:
######using power_transformer to transform asymmetric distribution curve to symmetric######
pt=PowerTransformer(method='yeo-johnson',standardize=True)

In [ ]:
x_train_pt=pt.fit_transform(x_train)

In [ ]:
x_test_pt=pt.transform(x_test)

In [ ]:
x_train_pt

In [ ]:
##standardization of the datasets
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [ ]:
x_train=scaler.fit_transform(x_train_pt)

In [ ]:
x_test=scaler.transform(x_test_pt)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model=LinearRegression()

In [ ]:
model.fit(x_train,y_train)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_slope=model.coef_
print(y_slope)

In [ ]:
y_intercept=model.intercept_
print(y_intercept)

In [ ]:
##predicted y_values
y_pred

In [ ]:
##calculating residual
residual=y_test-y_pred

In [ ]:
#assumptions of linear regression
plt.scatter(y_test,y_pred)
plt.xlabel("test truth data")
plt.ylabel("test predicted data")

In [ ]:
sns.displot(residual,kind="kde")

In [ ]:
#scatter plot with predictions and residual
plt.scatter(y_pred,residual)

In [ ]:
##calculating rmse,mae
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
print(mean_squared_error(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
print(np.sqrt(mean_squared_error(y_test,y_pred)))

In [ ]:
##r_square
from sklearn.metrics import r2_score
score=r2_score(y_test,y_pred)
print(score)

In [ ]:
##adjusted r_square
1 - (1-score)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)